In [2]:
from bs4 import BeautifulSoup
# import requests
from requests import get
# import json
import random
import string
import sqlite3
import logging
# import re


In [59]:
flickfilosopher_file = '../Scraping/flickfilosopher.txt'
current_reviews = set(line.strip() for line in open(flickfilosopher_file))

db_path = '../Data/test.db'
db_name = 'test'
conn = sqlite3.connect(db_path)
c = conn.cursor()

c.execute(("""
       CREATE TABLE IF NOT EXISTS
           test(
               NODE TEXT,
               URL TEXT,
               REVIEWBODY TEXT,
               RATING TEXT,
               REVIEWRATING TEXT,
               BESTRATING TEXT,
               WORSTRATING TEXT,
               PRIMARY KEY (NODE, URL))
   """))

In [60]:
def generateNode(length):
    letters_and_digits = string.ascii_letters + string.digits
    result_str = ''.join((random.choice(letters_and_digits) for i in range(length)))
    node = "_:znode" + result_str
    return node

review_count = 0
no_annotation = 0
no_rating = 0

# all moviereviews
# url = "https://www.flickfilosopher.com/all-reviews-a-m"
page2 = "https://www.flickfilosopher.com/all-reviews-n-z"
url_lst = ['https://www.flickfilosopher.com/all-reviews-a-m', 'https://www.flickfilosopher.com/all-reviews-n-z']
for url in url_lst:
    soup = BeautifulSoup(get(url).text, 'html.parser')
    movie_container = soup.find_all("div", class_="entry post clearfix")
    for link in movie_container[0].find_all("a"):
        review_link = link.get('href')
        # print(link.get('href'))
        if review_link is not None:
            if review_link.endswith('html'):
                if review_link in current_reviews:
                    pass
                else:
                    review_soup = BeautifulSoup(get(review_link).text, 'lxml')
                    review_html = review_soup.find("div", itemtype="http://schema.org/Review")
                    # print(review_html)
                    reviewBody = ""
                    if review_html is None:
                        print("Kein Review Annotation")
                        no_annotation += 1
                    else:
                        reviewBody_html = review_html.find_all("div", class_="reviewtext")
                        reviewRating = review_soup.find("div", itemtype="http://schema.org/Rating")
                        for item in reviewBody_html:
                            reviewBody += item.p.text

                        # # print(reviewRating_html)
                        # for j in reviewRating_html:
                        #     print(j.meta.content)
                        worstRating = reviewRating.find("meta", itemprop="worstRating")["content"]
                        bestRating = reviewRating.find("meta", itemprop="bestRating")["content"]
                        ratingValue = reviewRating.find("meta", itemprop="ratingValue")["content"]
                        if ratingValue == "":
                            print("kein Rating")
                            no_rating += 1
                        else:
                            # speichern
                            node = generateNode(31)
                            print('node:' + node)
                            print('url: '+ review_link)
                            print('reviewBody: ' + reviewBody)
                            print('worstRating: ' + worstRating)
                            print('bestRating: ' + bestRating)
                            print('ratingValue: ' + ratingValue)
                            c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node,review_link,reviewBody,str(reviewRating),ratingValue,bestRating,worstRating))
                            conn.commit()
                            review_count += 1

logging.debug("amount of reviews extracted: " + str(review_count))


#     ...

kein Rating
kein Rating
kein Rating
Kein Review Annotation
kein Rating
kein Rating
kein Rating
Kein Review Annotation


KeyboardInterrupt: 

In [ ]:
soup = BeautifulSoup(get(url).text, 'html.parser')
movie_container = soup.find_all("div", class_="entry post clearfix")
for link in movie_container[0].find_all("a"):
    review_link = link.get('href')
    # print(link.get('href'))
    if review_link is not None:
        if review_link.endswith('html'):
            if review_link in current_reviews:
                pass
            else:
                # print(review_link)
                ...
                # manche haben schema.org/Review nicht drin
                # review_count += 1
    #     if review_link.endswith('html') & review_link not in current_reviews:
    #         print(review_link)
    #     else:
    #         pass
    #             #nicht alle Einträge sind vollständig (fehlende Bewertung)
    #             # review_count += 1
    # else:
    #     pass

# print(movie_container)
# print(review_count)
# logging.debug
# print('amount of reviews extracted: ' + str(review_count))

In [58]:
# review und rating
# review_link ="https://www.flickfilosopher.com/2020/06/my-spy-movie-review-and-you-can-keep-him-little-girl-amazonprime.html"
#weder review noch rating
# review_link="https://www.flickfilosopher.com/2002/04/crush-and-my-wife-is-an-actress-ma-femme-est-une-actrice-review.html"
# review aber kein rating
# review_link ="https://www.flickfilosopher.com/2002/05/the-mystic-masseur-review.html"
review_link="https://www.flickfilosopher.com/2013/02/oscar-nominated-animated-shorts-85th-academy-awards-review.html"
review_soup = BeautifulSoup(get(review_link).text, 'lxml')
review_html = review_soup.find("div", itemtype="http://schema.org/Review")
# print(review_html)
reviewBody = ""
if review_html is None:
    print("Kein Review Annotation")
    no_annotation += 1
else:
    reviewBody_html = review_html.find_all("div", class_="reviewtext")
    reviewRating = review_soup.find("div", itemtype="http://schema.org/Rating")
    for item in reviewBody_html:
        reviewBody += item.p.text

    # # print(reviewRating_html)
    # for j in reviewRating_html:
    #     print(j.meta.content)
    worstRating = reviewRating.find("meta", itemprop="worstRating")["content"]
    bestRating = reviewRating.find("meta", itemprop="bestRating")["content"]
    ratingValue = reviewRating.find("meta", itemprop="ratingValue")["content"]
    if ratingValue == "":
        print("kein Rating")
        no_rating += 1
    else:
        # speichern
        node = generateNode(31)
        print('node:' + node)
        print('url: '+ review_link)
        print('reviewBody: ' + reviewBody)
        print('worstRating: ' + worstRating)
        print('bestRating: ' + bestRating)
        print('ratingValue: ' + ratingValue)
        c.execute(f"INSERT OR IGNORE INTO {db_name} (NODE, URL, REVIEWBODY, RATING, REVIEWRATING, BESTRATING, WORSTRATING) VALUES (?,?,?,?,?,?,?);",(node,review_link,reviewBody,str(reviewRating),ratingValue,bestRating,worstRating))
        conn.commit()
        review_count += 1

logging.debug("")



Kein Review Annotation
